In [2]:

import chromadb
chroma_client = chromadb.Client()

In [9]:
collection = chroma_client.create_collection(name="finding-me-rag-01")

InternalError: Collection [finding-me-rag-01] already exists

In [10]:
collection.add(
    documents=[
        "This is Viola Davis memoir",
        "Viola Davis was born and raised in South Carolina",
    ],
    ids=["id1", "id2"]
)

In [11]:
results = collection.query(
    query_texts=["Who is the author of the book?"],
    n_results=2
)
print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is Viola Davis memoir', 'Viola Davis was born and raised in South Carolina']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[1.2902677059173584, 1.7714648246765137]]}


In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI



In [1]:
from pypdf import PdfReader, PdfWriter
input_path = "./data_source/FindingMeMemoir.pdf"
output_path = "./data_source/FindingMeMemoirTrimmed.pdf"

reader = PdfReader(input_path)
writer = PdfWriter()
last_page_to_keep = 250
for i in range(len(reader.pages)):
    if i <= last_page_to_keep:
        writer.add_page(reader.pages[i])

with open(output_path, 'wb') as f:
    writer.write(f)
    


In [1]:
from pypdf import PdfReader, PdfWriter

input_path = "./data_source/FindingMeMemoirTrimmed.pdf"
output_path = "./data_source/FindingMeMemoirTrimmed1.pdf"
reader = PdfReader(input_path)
writer = PdfWriter()
removed_pages = [0, 1, 2, 4, 5, 250]
for i in range(len(reader.pages)):
    if i not in removed_pages:
        writer.add_page(reader.pages[i])

with open(output_path, 'wb') as f:
    writer.write(f)



In [76]:
loader = PyPDFLoader("./data_source/FindingMeMemoirTrimmed1.pdf")
# loader = PyPDFLoader("./data_source/Linah-MuhonjaImbariCV.pdf")
pages = loader.load()
print(len(pages))

245


In [84]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)
# print(f"Number of pages: {len(pages)} with {len(chunks)} chunks")
print(len(chunks))

530


In [50]:
llm = ChatOpenAI()

In [85]:
from langchain.chains import load_summarize_chain
summarization_chain = load_summarize_chain(llm=llm, chain_type="map_reduce")

In [86]:
result = summarization_chain.invoke(chunks)

In [87]:
result['output_text']

"The book follows the narrator's childhood experiences dealing with trauma, bullying, and discrimination, shaped by their family dynamics and struggles. It explores themes of resilience, self-discovery, and coping with trauma through poverty, abuse, and hardship. The narrator finds solace in acting, pursuing their dreams, navigating relationships, and ultimately finding joy and self-acceptance despite challenges in the industry and personal life. The story culminates in the speaker's journey of self-transformation, self-acceptance, and defiance of societal expectations, inspired by role models like Viola Davis and Cicely Tyson. Through vulnerability and inner strength, the narrator embraces their past, finds success, and highlights the importance of representation and self-love for Black women."